In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [4]:
from numpy import array
from numpy import asarray
from numpy import zeros
# The Natural Language Toolkit, or more commonly NLTK, is a suite of libraries and programs for symbolic and 
# statistical natural language processing for English written in the Python programming language.
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import re

#TQDM is a progress bar library with good support for nested loops and Jupyter/IPython notebooks.
from tqdm import tqdm

In [7]:
train= pd.read_csv("toxic_comments.csv")


In [8]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [9]:
import string
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

train['comment_text'] = train['comment_text'].apply(lambda x: remove_punct(x))


In [10]:
tokens = [word_tokenize(sen) for sen in train.comment_text]

In [11]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

In [12]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [13]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

In [ ]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens] 

In [19]:
result = [' '.join(sen) for sen in filtered_words] 

In [20]:
train['comment_text']=result

In [21]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww matches background colour im seemingly st...,0,0,0,0,0,0
2,000113f07ec002fd,hey man im really trying edit war guy constant...,0,0,0,0,0,0
3,0001b41b1c6bb37e,cant make real suggestions improvement wondere...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chance remember page thats,0,0,0,0,0,0


In [67]:
embed_size = 100 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

In [24]:
X = train["comment_text"]

y = toxic_comments[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [26]:
y1_train = y_train[["toxic"]].values
y1_test =  y_test[["toxic"]].values

# Second output
y2_train = y_train[["severe_toxic"]].values
y2_test =  y_test[["severe_toxic"]].values

# Third output
y3_train = y_train[["obscene"]].values
y3_test =  y_test[["obscene"]].values

# Fourth output
y4_train = y_train[["threat"]].values
y4_test =  y_test[["threat"]].values

# Fifth output
y5_train = y_train[["insult"]].values
y5_test =  y_test[["insult"]].values

# Sixth output
y6_train = y_train[["identity_hate"]].values
y6_test =  y_test[["identity_hate"]].values

In [28]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [33]:
embeddings_index = dict()
f = open(r'C:\Users\samar\glove.6B\glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [46]:
glove_file = open(r'C:\Users\samar\glove.6B\glove.6B.100d.txt', encoding="utf8")


In [47]:
embeddings_index = {}

for line in glove_file :
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [53]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

C:\Users\samar\anaconda3\envs\tf-gpu\lib\site-packages\IPython\core\interactiveshell.py:3337: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


(0.004451992, 0.4081574)

In [61]:
embedding_matrix = np.zeros((20000, 100))
for word, index in tokenizer.word_index.items():
    if index > 20000 - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [62]:
from keras.layers import Bidirectional, GlobalMaxPool1D


In [69]:
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(max_features, embed_size, weights=[embedding_matrix])(deep_inputs)

LSTM_Layer_1 = Bidirectional(LSTM(100, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(embedding_layer)
glob_layer = GlobalMaxPool1D()(LSTM_Layer_1)
dense_layer_1 = Dense(100, activation="relu")(glob_layer)
dense_layer_2 = Dropout(0.1)(dense_layer_1)
dense_layer_3 = Dense(1, activation="sigmoid")(dense_layer_2)
model = Model(inputs=deep_inputs, outputs=dense_layer_3)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [74]:
model2 = Model(inputs=deep_inputs, outputs=dense_layer_3)
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [70]:
model.fit(X_train, y1_train, batch_size=32, epochs=2, validation_split=0.1);


Epoch 1/2
3591/3591 [==============================] - 1085s 302ms/step - loss: 0.1191 - accuracy: 0.9564 - val_loss: 0.1006 - val_accuracy: 0.9621
Epoch 2/2
3591/3591 [==============================] - 674s 188ms/step - loss: 0.0877 - accuracy: 0.9672 - val_loss: 0.1035 - val_accuracy: 0.9616


In [71]:
score = model.evaluate(X_test, y1_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

998/998 [==============================] - 19s 19ms/step - loss: 0.1033 - accuracy: 0.9625
Test Score: 0.10330891609191895
Test Accuracy: 0.9624941349029541


In [72]:
model.fit(X_train, y2_train, batch_size=32, epochs=1, validation_split=0.1);


3591/3591 [==============================] - 535s 149ms/step - loss: 0.0232 - accuracy: 0.9902 - val_loss: 0.0236 - val_accuracy: 0.9893


In [73]:
score = model.evaluate(X_test, y2_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

998/998 [==============================] - 20s 20ms/step - loss: 0.0233 - accuracy: 0.9902
Test Score: 0.023334059864282608
Test Accuracy: 0.990192711353302


In [75]:
model2.fit(X_train, y3_train, batch_size=32, epochs=1, validation_split=0.1);


3591/3591 [==============================] - 541s 151ms/step - loss: 0.0494 - accuracy: 0.9813 - val_loss: 0.0552 - val_accuracy: 0.9809


In [76]:
score = model2.evaluate(X_test, y3_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

998/998 [==============================] - 21s 21ms/step - loss: 0.0495 - accuracy: 0.9814
Test Score: 0.04950924962759018
Test Accuracy: 0.9813880324363708


In [77]:
score = model2.evaluate(X_test, y4_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

998/998 [==============================] - 21s 21ms/step - loss: 0.1719 - accuracy: 0.9518
Test Score: 0.1719256043434143
Test Accuracy: 0.9517781734466553


In [78]:
score = model2.evaluate(X_test, y5_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

998/998 [==============================] - 21s 21ms/step - loss: 0.0957 - accuracy: 0.9668
Test Score: 0.09574627876281738
Test Accuracy: 0.9667554497718811


In [79]:
score = model2.evaluate(X_test, y6_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

998/998 [==============================] - 23s 23ms/step - loss: 0.1645 - accuracy: 0.9535
Test Score: 0.16453127562999725
Test Accuracy: 0.9535328149795532


In [ ]:
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(6, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])